In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path=r'C:\Users\jphil\OneDrive\Documents\CareerFoundry Data Analytics\Specialization- Machine Learning'

In [70]:
df=pd.read_csv(os.path.join(path, 'Achievement 1', 'Data Sets', 'Original Data', 'Dataset-weather-prediction-dataset-processed.csv'), index_col=False)

In [3]:
answers=pd.read_csv(os.path.join(path, 'Achievement 1', 'Data Sets', 'Pleasant_Weather_Answers.csv'), index_col=False)

# Data cleaning

In [72]:
df.head()

DATE  MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0  19600101      1                  7               2.1            0.85   
1  19600102      1                  6               2.1            0.84   
2  19600103      1                  8               2.1            0.90   
3  19600104      1                  3               2.1            0.92   
4  19600105      1                  6               2.1            0.95   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0           1.018                    0.32                 0.09   
1           1.018                    0.36                 1.05   
2           1.018                    0.18                 0.30   
3           1.018                    0.58                 0.00   
4           1.018                    0.65                 0.14   

   BASEL_snow_depth  BASEL_sunshine  ...  VALENTIA_cloud_cover  \
0                 0             0.7  ...                     5   
1                 0             1.1  ...                     7   
2                 0             0.0  ...                     7   
3                 0             4.1  ...                     7   
4                 0             5.4  ...                     3   

   VALENTIA_humidity  VALENTIA_pressure  VALENTIA_global_radiation  \
0               0.88             1.0003                       0.45   
1               0.91             1.0007                       0.25   
2               0.91             1.0096                       0.17   
3               0.86             1.0184                       0.13   
4               0.80             1.0328                       0.46   

   VALENTIA_precipitation  VALENTIA_snow_depth  VALENTIA_sunshine  \
0                    0.34                    0                4.7   
1                    0.84                    0                0.7   
2                    0.08                    0                0.1   
3                    0.98                    0                0.0   
4                    0.00                    0                5.7   

   VALENTIA_temp_mean  VALENTIA_temp_min  VALENTIA_temp_max  
0                 8.5                6.0               10.9  
1                 8.9                5.6               12.1  
2                10.5                8.1               12.9  
3                 7.4                7.3               10.6  
4                 5.7                3.0                8.4  

[5 rows x 170 columns]

In [73]:
#Drop the DATE and MONTH columns
df_dropped = df.drop(['DATE','MONTH'], axis=1)

In [4]:
answers=answers.drop(['DATE'], axis=1)

In [75]:
#dropping columns pertaining to the 3 stations not included in the answers dataset
df_drop=df_dropped.drop(columns=['GDANSK_cloud_cover','GDANSK_humidity', 'GDANSK_precipitation','GDANSK_snow_depth','GDANSK_temp_mean','GDANSK_temp_min','GDANSK_temp_max','ROMA_cloud_cover','ROMA_wind_speed','ROMA_humidity','ROMA_pressure','ROMA_sunshine','ROMA_temp_mean','TOURS_wind_speed','TOURS_humidity','TOURS_pressure','TOURS_global_radiation','TOURS_precipitation','TOURS_temp_mean','TOURS_temp_min','TOURS_temp_max'])

In [76]:
# Extract the different observation types

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [77]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in df_drop.columns if col.endswith(obs)]
    
    # Count the number of stations (i.e., the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")

Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


In [78]:
# Drop columns that end with wind_speed and snow_depth from the dataset

columns_to_drop = df_drop.filter(regex='(_wind_speed|_snow_depth)$').columns
columns_to_drop

Index(['BASEL_wind_speed', 'BASEL_snow_depth', 'DEBILT_wind_speed',
       'DUSSELDORF_wind_speed', 'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
       'KASSEL_wind_speed', 'LJUBLJANA_wind_speed', 'MAASTRICHT_wind_speed',
       'MADRID_wind_speed', 'MUNCHENB_snow_depth', 'OSLO_wind_speed',
       'OSLO_snow_depth', 'SONNBLICK_wind_speed', 'VALENTIA_snow_depth'],
      dtype='object')

In [79]:
unscaled = df_drop.drop(columns=columns_to_drop)

In [80]:
unscaled.shape

(22950, 132)

In [81]:
# Create a list of all unique station names in the dataset

all_stations = set([col.split('_')[0] for col in unscaled.columns if '_' in col])
all_stations

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

In [82]:
observation_types = ['cloud_cover', 'humidity', 'pressure']

missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in unscaled.columns if col.endswith(obs)]
    
    # Extract station names by removing the observation type from the column names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = all_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations missing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations missing from cloud_cover:
KASSEL

Stations missing from humidity:
STOCKHOLM

Stations missing from pressure:
MUNCHENB


In [83]:
# Get the position of HEATHROW_temp_max to see where we need to position the new KASSEL_cloud_cover  (+1 next to it)

unscaled.columns.get_loc('HEATHROW_temp_max')

53

In [84]:
unscaled.columns.get_loc('STOCKHOLM_cloud_cover') # +2

115

In [85]:
unscaled.columns.get_loc('MUNCHENB_humidity') # +2

90

In [86]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is copied from other existing columns:
# Kassel_cloud_cover with Dusseldorf_cloud_cover
# Stockholm_humidity with Oslo_humidity
# Munchenb_pressure with Basel_pressure

unscaled.insert(56,'KASSEL_cloud_cover', unscaled['DUSSELDORF_cloud_cover'])
unscaled.insert(119, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])
unscaled.insert(94,'MUNCHENB_pressure',unscaled['BASEL_pressure'])

In [87]:
unscaled.columns.tolist()

['BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_temp_mean',


In [88]:
unscaled.shape # observations dataset has the correct shape

(22950, 135)

In [89]:
answers.shape

(22950, 15)

In [90]:
#export cleaned dataset
unscaled.to_csv(os.path.join(path, 'Achievement 2', 'Data Sets', 'unscaled_cleaned.csv'), index=False)

In [5]:
X=pd.read_csv(os.path.join(path, 'Achievement 2', 'Data Sets', 'unscaled_cleaned.csv'), index_col=False)
X

BASEL_cloud_cover  BASEL_humidity  BASEL_pressure  \
0                      7            0.85          1.0180   
1                      6            0.84          1.0180   
2                      8            0.90          1.0180   
3                      3            0.92          1.0180   
4                      6            0.95          1.0180   
...                  ...             ...             ...   
22945                  1            0.79          1.0248   
22946                  6            0.77          1.0244   
22947                  4            0.76          1.0227   
22948                  5            0.80          1.0212   
22949                  5            0.84          1.0193   

       BASEL_global_radiation  BASEL_precipitation  BASEL_sunshine  \
0                        0.32                 0.09             0.7   
1                        0.36                 1.05             1.1   
2                        0.18                 0.30             0.0   
3                        0.58                 0.00             4.1   
4                        0.65                 0.14             5.4   
...                       ...                  ...             ...   
22945                    1.34                 0.22             7.7   
22946                    1.34                 0.22             5.4   
22947                    1.34                 0.22             6.1   
22948                    1.34                 0.22             5.8   
22949                    1.34                 0.22             3.2   

       BASEL_temp_mean  BASEL_temp_min  BASEL_temp_max  BELGRADE_cloud_cover  \
0                  6.5             0.8            10.9                     1   
1                  6.1             3.3            10.1                     6   
2                  8.5             5.1             9.9                     6   
3                  6.3             3.8            10.6                     8   
4                  3.0            -0.7             6.0                     8   
...                ...             ...             ...                   ...   
22945             15.9            11.4            21.4                     2   
22946             16.7            14.3            21.9                     0   
22947             16.7            13.1            22.4                     2   
22948             15.4            11.6            21.1                     1   
22949             13.5             9.9            19.2                     4   

       ...  STOCKHOLM_temp_max  VALENTIA_cloud_cover  VALENTIA_humidity  \
0      ...                 4.9                     5               0.88   
1      ...                 5.0                     7               0.91   
2      ...                 4.1                     7               0.91   
3      ...                 2.3                     7               0.86   
4      ...                 4.3                     3               0.80   
...    ...                 ...                   ...                ...   
22945  ...                14.2                     5               0.82   
22946  ...                14.3                     5               0.82   
22947  ...                14.4                     5               0.82   
22948  ...                12.4                     5               0.82   
22949  ...                12.6                     5               0.82   

       VALENTIA_pressure  VALENTIA_global_radiation  VALENTIA_precipitation  \
0                 1.0003                       0.45                    0.34   
1                 1.0007                       0.25                    0.84   
2                 1.0096                       0.17                    0.08   
3                 1.0184                       0.13                    0.98   
4                 1.0328                       0.46                    0.00   
...                  ...                        ...                     ...   
22945             1.0142      

In [6]:
y=answers

In [7]:
# Turn X and y from a df to arrays

X = np.array(X)
y = np.array(y)

In [8]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [9]:
X = X.reshape(-1,15,9)

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [11]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


# Creating Keras Model- 1st iteration

In [108]:
epochs = 50
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax')) # Don't use relu here!

In [109]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 7, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,423 (36.81 KB)

 Trainable params: 9,423 (36.81 KB)

 Non-trainable params: 0 (0.00 B)

In [110]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [111]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/50
1076/1076 - 7s - 7ms/step - accuracy: 0.0848 - loss: 10.2726
Epoch 2/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0903 - loss: 10.4779
Epoch 3/50
1076/1076 - 3s - 2ms/step - accuracy: 0.0885 - loss: 10.8066
Epoch 4/50
1076/1076 - 3s - 2ms/step - accuracy: 0.0900 - loss: 11.1712
Epoch 5/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0912 - loss: 11.6166
Epoch 6/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0919 - loss: 12.0646
Epoch 7/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0926 - loss: 12.5804
Epoch 8/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0869 - loss: 13.1556
Epoch 9/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0884 - loss: 13.5638
Epoch 10/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0831 - loss: 14.2317
Epoch 11/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0815 - loss: 14.7475
Epoch 12/50
1076/1076 - 4s - 3ms/step - accuracy: 0.0774 - loss: 15.1589
Epoch 13/50
1076/1076 - 4s - 3ms/step - accuracy: 0.0762 - loss: 15.8234
Epoch 14/50
1076/1076 - 3s - 3ms/step - accuracy: 0.0790 - l

In [112]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [113]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [114]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DUSSELDORF  LJUBLJANA  MADRID  \
True                                                                   
BASEL           0        17         4           0          0    1448   
BELGRADE        0        14         4           2          0     991   
BUDAPEST        0         4         0           0          1     208   
DEBILT          0         0         0           0          0      82   
DUSSELDORF      0         1         0           0          0      28   
HEATHROW        0         2         1           0          0      75   
KASSEL          0         0         0           0          0      11   
LJUBLJANA       0         1         1           0          0      59   
MAASTRICHT      0         0         0           1          0       5   
MADRID          1        12         1           0          0     251   
MUNCHENB        0         0         1           0          0       5   
OSLO            0      

# 2nd iteration- increased hidden layer, reduced dropout rate

In [115]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Parameters
epochs = 50
batch_size = 16
n_hidden = 64  # Increased number of hidden units for LSTM layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, return_sequences=True))  # First LSTM layer with return_sequences=True
model.add(BatchNormalization())  # Batch Normalization after first LSTM layer
model.add(LSTM(n_hidden))  # Second LSTM layer
model.add(Dropout(0.3))  # Reduced Dropout rate for more information retention
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.0693 - loss: 10.0177 - val_accuracy: 0.0514 - val_loss: 8.8108
Epoch 2/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0791 - loss: 9.7275 - val_accuracy: 0.0666 - val_loss: 8.8550
Epoch 3/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.0823 - loss: 9.9339 - val_accuracy: 0.0462 - val_loss: 8.9797
Epoch 4/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.0762 - loss: 9.8951 - val_accuracy: 0.0570 - val_loss: 8.8617
Epoch 5/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.0733 - loss: 9.9213 - val_accuracy: 0.0474 - val_loss: 8.9848
Epoch 6/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.0796 - loss: 10.0504 - val_accuracy: 0.0638 - val_loss: 9.0497
Epoch 7/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.0847 - loss: 10.2464 - val_accuracy: 0.0582 - val_loss: 9.1566
Epoch 8/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.0902 - loss: 10.15

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_9 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 7, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 7, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,864 (534.63 KB)

 Trainable params: 68,367 (267.06 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 68,369 (267.07 KB)

# 3rd iteration- reduced learning rate and increased hidden layers

In [116]:
# Parameters
epochs = 50
batch_size = 32
n_hidden = 128  # Increased number of hidden units for LSTM layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, return_sequences=True))  # First LSTM layer with return_sequences=True
model.add(BatchNormalization())  # Batch Normalization after first LSTM layer
model.add(LSTM(n_hidden))  # Second LSTM layer
model.add(Dropout(0.3))  # Reduced Dropout rate for more information retention
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.1487 - loss: 9.5006 - val_accuracy: 0.1987 - val_loss: 8.8645
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1546 - loss: 9.9379 - val_accuracy: 0.2049 - val_loss: 8.6781
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1403 - loss: 10.0835 - val_accuracy: 0.2070 - val_loss: 8.5737
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1320 - loss: 9.7863 - val_accuracy: 0.1934 - val_loss: 8.5626
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1165 - loss: 9.8130 - val_accuracy: 0.1806 - val_loss: 8.5779
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1154 - loss: 9.7432 - val_accuracy: 0.2022 - val_loss: 8.5307
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1019 - loss: 9.8047 - val_accuracy: 0.0559 - val_loss: 8.5703
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0957 - loss: 9.6219 - val_accur

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 7, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 7, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,840 (2.04 MB)

 Trainable params: 267,791 (1.02 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 267,793 (1.02 MB)

# 4th iteration- reduced learning rate and increased dropout slightly

In [117]:
# Parameters
epochs = 50
batch_size = 32
n_hidden = 128  # Increased number of hidden units for LSTM layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, return_sequences=True))  # First LSTM layer with return_sequences=True
model.add(BatchNormalization())  # Batch Normalization after first LSTM layer
model.add(LSTM(n_hidden))  # Second LSTM layer
model.add(Dropout(0.4))  
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.0946 - loss: 9.4576 - val_accuracy: 0.1201 - val_loss: 9.0138
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1272 - loss: 10.7831 - val_accuracy: 0.0889 - val_loss: 8.9453
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1095 - loss: 10.5007 - val_accuracy: 0.0901 - val_loss: 8.9021
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1053 - loss: 10.8117 - val_accuracy: 0.0662 - val_loss: 8.9319
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1065 - loss: 10.6651 - val_accuracy: 0.0816 - val_loss: 8.7080
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1086 - loss: 10.3661 - val_accuracy: 0.0695 - val_loss: 8.7478
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1050 - loss: 10.6574 - val_accuracy: 0.0819 - val_loss: 8.7139
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.1006 - loss: 10.6271 - val_a

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_11 (Conv1D)              │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 7, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 7, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,840 (2.04 MB)

 Trainable params: 267,791 (1.02 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 267,793 (1.02 MB)

# 5th iteration- trying a GRU-based approach over LSTM

In [118]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Parameters
epochs = 50
batch_size = 32
n_hidden = 128  # Number of hidden units in GRU layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(GRU(n_hidden, return_sequences=True))  # First GRU layer with return_sequences=True
model.add(BatchNormalization())  # Batch Normalization after the first GRU layer
model.add(GRU(n_hidden))  # Second GRU layer
model.add(Dropout(0.4))  # Dropout for regularization
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0983 - loss: 9.5708 - val_accuracy: 0.0756 - val_loss: 8.7645
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1190 - loss: 10.5282 - val_accuracy: 0.1509 - val_loss: 8.7864
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.1199 - loss: 10.5915 - val_accuracy: 0.0647 - val_loss: 8.7724
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1136 - loss: 10.6375 - val_accuracy: 0.0657 - val_loss: 8.7549
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1126 - loss: 10.5964 - val_accuracy: 0.0676 - val_loss: 8.7264
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1068 - loss: 10.3270 - val_accuracy: 0.0695 - val_loss: 8.7204
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1153 - loss: 10.6277 - val_accuracy: 0.0718 - val_loss: 8.7015
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.1071 - loss: 10.6044 - va

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_12 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 7, 128)         │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 7, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 405,792 (1.55 MB)

 Trainable params: 202,767 (792.06 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 202,769 (792.07 KB)

# 6th iteration- using a bidirectional GRU- best-performing iteration (post-analysis)

In [122]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Parameters
epochs = 50
batch_size = 32
n_hidden = 128  # Number of hidden units in GRU layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(Bidirectional(GRU(n_hidden, return_sequences=True)))  # First Bidirectional GRU layer
model.add(BatchNormalization())  # Batch Normalization after first GRU layer
model.add(Bidirectional(GRU(n_hidden)))  # Second Bidirectional GRU layer
model.add(Dropout(0.4))  # Dropout for regularization
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - accuracy: 0.1477 - loss: 10.2904 - val_accuracy: 0.2266 - val_loss: 9.8232
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1572 - loss: 11.1463 - val_accuracy: 0.2232 - val_loss: 9.6155
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.1379 - loss: 11.3286 - val_accuracy: 0.2116 - val_loss: 9.5129
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.1267 - loss: 11.2308 - val_accuracy: 0.0655 - val_loss: 9.4984
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.1151 - loss: 11.3047 - val_accuracy: 0.0725 - val_loss: 9.4939
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.1139 - loss: 11.1681 - val_accuracy: 0.0723 - val_loss: 9.4778
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.1067 - loss: 11.2928 - val_accuracy: 0.0674 - val_loss: 9.5015
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.1086 - loss: 11.3293 -

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 7, 256)         │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 7, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,003,296 (3.83 MB)

 Trainable params: 501,391 (1.91 MB)

 Non-trainable params: 512 (2.00 KB)

 Optimizer params: 501,393 (1.91 MB)

In [123]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [124]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Pred        BELGRADE  BUDAPEST  DEBILT  KASSEL  MADRID  SONNBLICK  VALENTIA
True                                                                       
BASEL             33         0       1       0    1447       1856       345
BELGRADE          79         0       0       3     962         12        36
BUDAPEST          11         0       0       0     199          0         4
DEBILT             0         0       0       0      82          0         0
DUSSELDORF         0         0       1       0      28          0         0
HEATHROW           1         0       1       0      75          0         5
KASSEL             0         0       0       0      11          0         0
LJUBLJANA          1         1       0       0      58          0         1
MAASTRICHT         0         0       0       0       5          0         4
MADRID             3         0       0       0     335          9       111
MUNCHENB           0         0       0       0 

# 7th iteration- adding another Conv1D layer

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Parameters
epochs = 50
batch_size = 32
n_hidden = 128  # Number of hidden units in GRU layers

# Determine input shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=3, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(Conv1D(n_hidden, kernel_size=3, activation='relu'))  # Added second Conv1D layer
model.add(MaxPooling1D())
model.add(Bidirectional(GRU(n_hidden, return_sequences=True)))  # First Bidirectional GRU layer
model.add(BatchNormalization())  # Batch Normalization after first GRU layer
model.add(Bidirectional(GRU(n_hidden)))  # Second Bidirectional GRU layer
model.add(Dropout(0.4))  # Dropout for regularization
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.0853 - loss: 9.5592 - val_accuracy: 0.0953 - val_loss: 9.2440
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.1010 - loss: 10.8867 - val_accuracy: 0.0796 - val_loss: 9.2077
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.0846 - loss: 10.9232 - val_accuracy: 0.0760 - val_loss: 9.1554
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.0825 - loss: 11.0757 - val_accuracy: 0.0664 - val_loss: 9.1655
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0837 - loss: 10.9097 - val_accuracy: 0.0708 - val_loss: 9.1531
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.0869 - loss: 10.8788 - val_accuracy: 0.0781 - val_loss: 9.1345
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0785 - loss: 10.9760 - val_accuracy: 0.0735 - val_loss: 9.1356
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.0859 - loss: 11.0196 - 

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 13, 128)        │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 4, 128)         │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 2, 256)         │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 2, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,104,160 (4.21 MB)

 Trainable params: 551,823 (2.11 MB)

 Non-trainable params: 512 (2.00 KB)

 Optimizer params: 551,825 (2.11 MB)

# 8th iteration- reducing model complexity

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Parameters
epochs = 50
batch_size = 32
n_hidden = 64  # Reduced number of hidden units in GRU layers

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=3, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(Conv1D(n_hidden, kernel_size=3, activation='relu'))
model.add(MaxPooling1D())
model.add(Bidirectional(GRU(n_hidden, return_sequences=True)))  # First Bidirectional GRU layer with reduced units
model.add(BatchNormalization())  # Batch Normalization after first GRU layer
model.add(Bidirectional(GRU(n_hidden)))  # Second Bidirectional GRU layer with reduced units
model.add(Dropout(0.4))  # Dropout for regularization
model.add(Dense(n_classes, activation='softmax'))  # Softmax for multi-class classification

# Compile the model with a lower learning rate
model.compile(optimizer=RMSprop(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Model summary (optional, for reference)
model.summary()

Epoch 1/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.1119 - loss: 9.3569 - val_accuracy: 0.0620 - val_loss: 9.1331
Epoch 2/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0970 - loss: 10.7007 - val_accuracy: 0.0746 - val_loss: 9.2013
Epoch 3/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0830 - loss: 10.9116 - val_accuracy: 0.0681 - val_loss: 9.2097
Epoch 4/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0831 - loss: 10.8203 - val_accuracy: 0.0680 - val_loss: 9.1872
Epoch 5/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0789 - loss: 10.8490 - val_accuracy: 0.0678 - val_loss: 9.1495
Epoch 6/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0868 - loss: 10.9058 - val_accuracy: 0.0783 - val_loss: 9.0827
Epoch 7/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0818 - loss: 10.8718 - val_accuracy: 0.0711 - val_loss: 9.0850
Epoch 8/50
538/538 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0840 - loss: 11.0450 - val_accu

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)              │ (None, 13, 64)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_16 (MaxPooling1D) │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 4, 64)          │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 2, 128)         │        49,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 281,760 (1.07 MB)

 Trainable params: 140,751 (549.81 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 140,753 (549.82 KB)

# 9th iteration (first one post-feedback): reduced learning rate with class weights

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Parameters
epochs = 50
batch_size = 32
learning_rate = 0.0005

# Reshape y if necessary (ensure y is 1D for class weight calculation)
y = np.argmax(y, axis=1) if y.ndim > 1 else y

# Calculate class weights to handle class imbalance
unique_classes = np.unique(y)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y)
class_weights_dict = dict(enumerate(class_weights))

# Define the RNN model
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(15, 9)))
model.add(Dropout(0.3))  # Dropout for regularization
model.add(LSTM(32, activation='relu'))  # Second LSTM layer
model.add(Dropout(0.3))
model.add(Dense(15, activation='softmax'))  # Set the number of units to 15

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val), class_weight=class_weights_dict, 
                    callbacks=[early_stopping])

# Evaluate model performance on the validation set
validation_loss, validation_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

Epoch 1/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.2640 - loss: 2.6274 - val_accuracy: 0.2898 - val_loss: 1.9990
Epoch 2/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3353 - loss: 2.2308 - val_accuracy: 0.3179 - val_loss: 1.9162
Epoch 3/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.3429 - loss: 2.2607 - val_accuracy: 0.3621 - val_loss: 1.6526
Epoch 4/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.3666 - loss: 2.0102 - val_accuracy: 0.4333 - val_loss: 1.5771
Epoch 5/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.3821 - loss: 2.1623 - val_accuracy: 0.4510 - val_loss: 1.5288
Epoch 6/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.3879 - loss: 1.8870 - val_accuracy: 0.4414 - val_loss: 1.6219
Epoch 7/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.3795 - loss: 2.0066 - val_accuracy: 0.4719 - val_loss: 1.5014
Epoch 8/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.4128 - loss: 1.8675 - val_accura

# 10th iteration: further reduction in learning rate, addition of LSTM layer and random oversampling

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
import numpy as np

# Parameters
epochs = 50
batch_size = 32
learning_rate = 0.0001  # Smaller learning rate for stability

# Reshape y if necessary (ensure y is 1D for oversampling)
y = np.argmax(y, axis=1) if y.ndim > 1 else y

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X_train for oversampling (requires 2D format for sampling)
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Apply Random Oversampling to handle class imbalance
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_flat, y_train)

# Reshape X_train_resampled back to original 3D shape for RNN input
X_train_resampled = X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])

# Define the enhanced RNN model
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(15, 9)))  # Increased units
model.add(Dropout(0.3))
model.add(LSTM(64, activation='relu', return_sequences=True))  # Additional LSTM layer
model.add(Dropout(0.3))
model.add(LSTM(32, activation='relu'))  # Final LSTM layer
model.add(Dropout(0.3))
model.add(Dense(15, activation='softmax'))  # Output layer with 15 units

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with the balanced data
history = model.fit(X_train_resampled, y_train_resampled, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate model performance on the validation set
validation_loss, validation_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

Epoch 1/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 121s 22ms/step - accuracy: 0.2378 - loss: 2.2788 - val_accuracy: 0.5612 - val_loss: 1.2894
Epoch 2/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 125s 24ms/step - accuracy: 0.5866 - loss: 1.2211 - val_accuracy: 0.6290 - val_loss: 1.2188
Epoch 3/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 117s 23ms/step - accuracy: 0.7004 - loss: 0.8741 - val_accuracy: 0.6575 - val_loss: 1.0949
Epoch 4/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 115s 22ms/step - accuracy: 0.7600 - loss: 0.6917 - val_accuracy: 0.6553 - val_loss: 1.0841
Epoch 5/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 117s 23ms/step - accuracy: 0.7985 - loss: 0.5843 - val_accuracy: 0.6658 - val_loss: 1.0922
Epoch 6/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 116s 22ms/step - accuracy: 0.8270 - loss: 0.4988 - val_accuracy: 0.6717 - val_loss: 1.0471
Epoch 7/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 118s 23ms/step - accuracy: 0.8522 - loss: 0.4291 - val_accuracy: 0.6686 - val_loss: 1.1360
Epoch 8/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 115s 22ms/step - accuracy: 

In [17]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Generate Predictions
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_val  # Ensure y_val is in integer label format

# Step 2: Create Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Optional: Visualize Confusion Matrix as Heatmap with Station Labels
station_labels = ['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 'HEATHROW', 'KASSEL', 
                  'LJUBLJANA', 'MAASTRICHT', 'MADRID', 'MUNCHENB', 'OSLO', 'SONNBLICK', 
                  'STOCKHOLM', 'VALENTIA']  # Update with actual station names if needed

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=station_labels, yticklabels=station_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Identify unique classes in y_true
unique_classes = np.unique(y_true)
print("Unique classes in y_true:", unique_classes)

# Filter station labels to match the unique classes in y_true
filtered_station_labels = [station_labels[i] for i in unique_classes]

# Generate the classification report using the filtered labels
class_report = classification_report(y_true, y_pred_classes, target_names=filtered_station_labels)
print("Classification Report:\n", class_report)

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Confusion Matrix:
 [[2260  342  113   55   23   39    6   25    3   78    5    0    1    5]
 [  70  551  179   18    6   20    0    9    0   24    0    1    1    0]
 [   6   41   86   10    6    6    1    4    0    2    0    0    0    0]
 [   3    1   15   31    9    3    2    0    0    0    0    0    0    0]
 [   0    1    8    4    8    3    1    0    0    0    0    0    0    0]
 [   2    0    7    2    2   41    0    1    1   10    0    0    0    1]
 [   0    3    2    0    0    1    0    2    0    1    0    0    0    0]
 [   0    5   10    0    0    7    1   20    0    3    0    0    0    0]
 [   1    1    0    0    3    1    0    0    0    1    0    0    0    0]
 [   4    7   38    1    4   21    0   10    0  274    1    0    0    0]
 [   0    0    1    0    0    0    0    1    0    3    2    0    0    1]
 [   0    0    1    0    0    1    0    1    0    0    0    1    0    0]
 [   0    1    1    0    0    1    0    0    0    0    0    0   

Unique classes in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Classification Report:
               precision    recall  f1-score   support

       BASEL       0.96      0.76      0.85      2955
    BELGRADE       0.58      0.63      0.60       879
    BUDAPEST       0.19      0.53      0.28       162
      DEBILT       0.26      0.48      0.34        64
  DUSSELDORF       0.13      0.32      0.19        25
    HEATHROW       0.28      0.61      0.39        67
      KASSEL       0.00      0.00      0.00         9
   LJUBLJANA       0.27      0.43      0.34        46
  MAASTRICHT       0.00      0.00      0.00         7
      MADRID       0.69      0.76      0.72       360
    MUNCHENB       0.25      0.25      0.25         8
        OSLO       0.50      0.25      0.33         4
   STOCKHOLM       0.00      0.00      0.00         3
    VALENTIA       0.12      1.00      0.22         1

    accuracy                           0.71      4590
   macro avg       0.30      0.43      0

# 11th iteration- assigning class weights to the loss function

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Parameters
epochs = 50
batch_size = 32
learning_rate = 0.0001

# Reshape y if necessary (ensure y is 1D for class weight calculation)
y = np.argmax(y, axis=1) if y.ndim > 1 else y

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate class weights to handle class imbalance
unique_classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Define the RNN model with LSTM layers
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(15, 9)))  # Increased units
model.add(Dropout(0.3))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(15, activation='softmax'))

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val), class_weight=class_weights_dict, 
                    callbacks=[early_stopping])

# Evaluate model performance on the validation set
validation_loss, validation_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

Epoch 1/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.2233 - loss: 2.4580 - val_accuracy: 0.4558 - val_loss: 1.7741
Epoch 2/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.3704 - loss: 2.3210 - val_accuracy: 0.3519 - val_loss: 2.0007
Epoch 3/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.3444 - loss: 2.2370 - val_accuracy: 0.3575 - val_loss: 1.7802
Epoch 4/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.3578 - loss: 2.0843 - val_accuracy: 0.3277 - val_loss: 1.8928
Epoch 5/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.3541 - loss: 2.0526 - val_accuracy: 0.4327 - val_loss: 1.6042
Epoch 6/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.3731 - loss: 2.1987 - val_accuracy: 0.4423 - val_loss: 1.6481
Epoch 7/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.3772 - loss: 2.0633 - val_accuracy: 0.4264 - val_loss: 1.5898
Epoch 8/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.3932 - loss: 2.0115 - v

# 12th iteration- random oversampling and enhanced architecture

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import RandomOverSampler
import numpy as np

# Parameters
epochs = 50
batch_size = 32
learning_rate = 0.0001  # Smaller learning rate for stability

# Reshape y if necessary (ensure y is 1D for oversampling)
y = np.argmax(y, axis=1) if y.ndim > 1 else y

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X_train for oversampling (requires 2D format for sampling)
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Apply Random Oversampling to handle class imbalance
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_flat, y_train)

# Reshape X_train_resampled back to original 3D shape for RNN input
X_train_resampled = X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])

# Define the RNN model with LSTM layers
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(15, 9)))  # Increased units
model.add(Dropout(0.3))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(15, activation='softmax'))

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with the balanced data
history = model.fit(X_train_resampled, y_train_resampled, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate model performance on the validation set
validation_loss, validation_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

Epoch 1/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 110s 20ms/step - accuracy: 0.2355 - loss: 2.3037 - val_accuracy: 0.5176 - val_loss: 1.3592
Epoch 2/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 120s 23ms/step - accuracy: 0.5969 - loss: 1.2039 - val_accuracy: 0.5941 - val_loss: 1.2697
Epoch 3/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 113s 22ms/step - accuracy: 0.7074 - loss: 0.8538 - val_accuracy: 0.6447 - val_loss: 1.1350
Epoch 4/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 112s 22ms/step - accuracy: 0.7537 - loss: 0.7087 - val_accuracy: 0.6444 - val_loss: 1.1500
Epoch 5/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 110s 21ms/step - accuracy: 0.7854 - loss: 0.6129 - val_accuracy: 0.6625 - val_loss: 1.1024
Epoch 6/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 113s 22ms/step - accuracy: 0.8149 - loss: 0.5302 - val_accuracy: 0.6654 - val_loss: 1.0736
Epoch 7/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 119s 23ms/step - accuracy: 0.8376 - loss: 0.4710 - val_accuracy: 0.6769 - val_loss: 1.1079
Epoch 8/50
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 117s 23ms/step - accuracy: 

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Generate Predictions
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_val  # Ensure y_val is in integer label format

# Step 2: Create Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Optional: Visualize Confusion Matrix as Heatmap with Station Labels
station_labels = ['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 'HEATHROW', 'KASSEL', 
                  'LJUBLJANA', 'MAASTRICHT', 'MADRID', 'MUNCHENB', 'OSLO', 'SONNBLICK', 
                  'STOCKHOLM', 'VALENTIA']  # Update with actual station names if needed

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=station_labels, yticklabels=station_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Identify unique classes in y_true
unique_classes = np.unique(y_true)
print("Unique classes in y_true:", unique_classes)

# Filter station labels to match the unique classes in y_true
filtered_station_labels = [station_labels[i] for i in unique_classes]

# Generate the classification report with the filtered labels
class_report = classification_report(y_true, y_pred_classes, target_names=filtered_station_labels)
print("Classification Report:\n", class_report)

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Confusion Matrix:
 [[2375  249   93   72   29   31    3   22    2   70    7    0    1    1]
 [ 114  536  137   36    5   21    1   10    0   17    1    1    0    0]
 [   5   41   74   22    4    6    0    4    0    6    0    0    0    0]
 [   4    1   12   36    4    6    0    0    0    0    0    0    0    1]
 [   0    0    5    7    6    6    0    1    0    0    0    0    0    0]
 [   2    1    9    6    3   33    3    0    1    9    0    0    0    0]
 [   0    2    2    1    3    0    0    0    0    1    0    0    0    0]
 [   0    7   11    0    0    4    1   18    0    5    0    0    0    0]
 [   2    2    1    0    2    0    0    0    0    0    0    0    0    0]
 [  11   10   32    2    5   25    0    9    0  266    0    0    0    0]
 [   2    0    2    0    0    1    0    1    0    2    0    0    0    0]
 [   0    0    2    0    0    0    0    0    0    1    0    1    0    0]
 [   0    0    0    1    1    0    0    0    0    1    0    0   

Unique classes in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Classification Report:
               precision    recall  f1-score   support

       BASEL       0.94      0.80      0.87      2955
    BELGRADE       0.63      0.61      0.62       879
    BUDAPEST       0.19      0.46      0.27       162
      DEBILT       0.20      0.56      0.29        64
  DUSSELDORF       0.10      0.24      0.14        25
    HEATHROW       0.25      0.49      0.33        67
      KASSEL       0.00      0.00      0.00         9
   LJUBLJANA       0.28      0.39      0.32        46
  MAASTRICHT       0.00      0.00      0.00         7
      MADRID       0.70      0.74      0.72       360
    MUNCHENB       0.00      0.00      0.00         8
        OSLO       0.50      0.25      0.33         4
   STOCKHOLM       0.00      0.00      0.00         3
    VALENTIA       0.33      1.00      0.50         1

    accuracy                           0.73      4590
   macro avg       0.29      0.40      0